# Data Engineer Certification - Practical Exam - Supplement Experiments

1001-Experiments makes personalized supplements tailored to individual health needs.

1001-Experiments aims to enhance personal health by using data from wearable devices and health apps.

This data, combined with user feedback and habits, is used to analyze and refine the effectiveness of the supplements provided to the user through multiple small experiments.

The data engineering team at 1001-Experiments plays a crucial role in ensuring the collected health and activity data from thousands of users is accurately organized and integrated with the data from supplement usage. 

This integration helps 1001-Experiments provide more targeted health and wellness recommendations and improve supplement formulations.


## Task

1001-Experiments currently has the following four datasets with four months of data:
 - "user_health_data.csv" which logs daily health metrics, habits and data from wearable devices,
 - "supplement_usage.csv" which records details on supplement intake per user,
 - "experiments.csv" which contains metadata on experiments, and
 - "user_profiles.csv" which contains demographic and contact information of the users.

Each dataset contains unique identifiers for users and/or their supplement regimen.

The developers and data scientsits currently manage code that cross-references all of these data sources separately, which is cumbersome and error-prone.

Your manager has asked you to write a Python function that cleans and merges these datasets into a single dataset.

The final dataset should provide a comprehensive view of each user's health metrics, supplement usage, and demographic information.

- To test your code, your manager will run only the code `merge_all_data('user_health_data.csv', 'supplement_usage.csv', 'experiments.csv', 'user_profiles.csv')`
- Your `merge_all_data` function must return a DataFrame, with columns as described below.
- All columns must accurately match the descriptions provided below, including names.


## Data

The provided data is structured as follows:

![database schema](schema.png)

The function you write should return data as described below.

There should be a unique row for each daily entry combining health metrics and supplement usage.

Where missing values are permitted, they should be in the default Python format unless stated otherwise.

| Column Name        | Description |
|--------------------|-------------|
| user_id            | Unique identifier for each user. </br>There should not be any missing values. |
| date               | The date the health data was recorded or the supplement was taken, in date format. </br>There should not be any missing values. |
| email              | Contact email of the user. </br>There should not be any missing values. |
| user_age_group  | The age group of the user, one of: 'Under 18', '18-25', '26-35', '36-45', '46-55', '56-65', 'Over 65' or 'Unknown' where the age is missing.|
| experiment_name    | Name of the experiment associated with the supplement usage. </br>Missing values for users that have user health data only is permitted. |
| supplement_name    | The name of the supplement taken on that day. Multiple entries are permitted. </br>Days without supplement intake should be encoded as 'No intake'. |
| dosage_grams       | The dosage of the supplement taken in grams. Where the dosage is recorded in mg it should be converted by division by 1000.</br>Missing values for days without supplement intake are permitted. |
| is_placebo         | Indicator if the supplement was a placebo (true/false). </br>Missing values for days without supplement intake are permitted. |
| average_heart_rate | Average heart rate as recorded by the wearable device. </br>Missing values are permitted. |
| average_glucose    | Average glucose levels as recorded on the wearable device. </br>Missing values are permitted. |
| sleep_hours        | Total sleep in hours for the night preceding the current day’s log. </br>Missing values are permitted. |
| activity_level     | Activity level score between 0-100. </br>Missing values are permitted. |

In [6]:
!pip install pandas
!pip install numpy

     ---------------------------------------- 11.6/11.6 MB 6.4 MB/s eta 0:00:00
     -------------------------------------- 346.6/346.6 KB 5.3 MB/s eta 0:00:00
     ---------------------------------------- 15.9/15.9 MB 7.9 MB/s eta 0:00:00
     -------------------------------------- 508.0/508.0 KB 6.4 MB/s eta 0:00:00


You should consider upgrading via the 'C:\Users\mines\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'C:\Users\mines\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip' command.


In [7]:
# Use as many python cells as you wish to write your code
import pandas as pd
import numpy as np

In [8]:
def merge_all_data(user_health_data_path, supplement_usage_path, experiments_path, user_profiles_path):
    
    # Clean sleep_hours to ensure they are in '9H' format
    def clean_sleep_hours(sleep):
        if pd.isna(sleep):
            return sleep
        elif 'h' in sleep.lower():
            return sleep.replace('h', 'H').strip()
        return sleep

    # Function to enforce a datatype on a column
    def enforceType(df, column, dtype, should_drop, fill_value=None):
        # Clean the column 
        df[column] = df[column].astype(str).str.strip()

        # Convert non-numeric values to NaN and enforce the numberic dtype as necessary
        if dtype in ['int32', 'Int32']:
            df[column] = pd.to_numeric(df[column], errors='coerce')
            if fill_value is not None:
                df[column].fillna(fill_value, inplace=True)
        #if should_drop == True:
         #   df.dropna(subset=[column], inplace=True)
        # Convert the column to specific dtype
        df[column] = df[column].astype(dtype)
        return df

    # Load the datasets
    user_profiles = pd.read_csv(user_profiles_path)
    user_health_data = pd.read_csv(user_health_data_path)
    supplement_usage = pd.read_csv(supplement_usage_path)
    experiments = pd.read_csv(experiments_path) 
    experiments.rename(columns={'name': 'experiment_name'}, inplace=True)

    ## Interpret database schema and combine multiple tables by rows and columns
    # Convert the date columns to datetime format
    user_health_data['date'] = pd.to_datetime(user_health_data['date'], errors='coerce')
    supplement_usage['date'] = pd.to_datetime(supplement_usage['date'], errors='coerce')

    # Enforce dtype
    user_profiles = enforceType(user_profiles, 'user_id', 'string', True, fill_value='null')
    user_profiles = enforceType(user_profiles, 'email', 'string', True, fill_value='null')
    #user_profiles = enforceType(user_profiles, 'age', 'int32', True, fill_value='null')

    user_health_data = enforceType(user_health_data, 'user_id', 'string', True, fill_value='null')
    user_health_data = enforceType(user_health_data, 'sleep_hours', 'string', False, fill_value='null')
    user_health_data = enforceType(user_health_data, 'average_heart_rate', 'float64', False, fill_value='null')
    user_health_data = enforceType(user_health_data, 'average_glucose', 'float64', False, fill_value='null')
    user_health_data = enforceType(user_health_data, 'activity_level', 'int32', False, fill_value='null')

    experiments = enforceType(experiments, 'experiment_id', 'string', False, fill_value='null')
    experiments = enforceType(experiments, 'experiment_name', 'string', False, fill_value='null')
    experiments = enforceType(experiments, 'description', 'string', False, fill_value='null')

    supplement_usage = enforceType(supplement_usage, 'user_id', 'string', False, fill_value='null')
    #supplement_usage = supplement_usage(result_df, 'date', 'datetime64', False, fill_value='null')
    supplement_usage = enforceType(supplement_usage, 'supplement_name', 'string', False, fill_value='null')
    #supplement_usage = supplement_usage(result_df, 'dosage', 'float64', False, fill_value='null')
    supplement_usage = enforceType(supplement_usage, 'dosage_unit', 'string', False, fill_value='null')
    #supplement_usage = enforceType(supplement_usage, 'is_placebo', 'string', False, fill_value='null')
    supplement_usage = enforceType(supplement_usage, 'experiment_id', 'string', False, fill_value='null')

    # Merge user_profiles with user_health_data
    merged_df = pd.merge(user_health_data, user_profiles[['user_id', 'email', 'age']], on='user_id', how='left')

    # Merge with supplement_usage
    merged_df = pd.merge(merged_df, supplement_usage, on=['user_id', 'date'], how='left')

    # Merge with experiments
    merged_df = pd.merge(merged_df, experiments, on=['experiment_id'], how='left')

    # Create age groups
    def age_group(age):

        if pd.isna(age):
            return 'Unknown'
        elif age < 18:
            return 'Under 18'
        elif 18 <= age <= 25:
            return '18-25'
        elif 26 <= age <= 35:
            return '26-35'
        elif 36 <= age <= 45:
            return '36-45'
        elif 46 <= age <= 55:
            return '46-55'
        elif 56 <= age <= 65:
            return '56-65'
        else:
            return 'Over 65'

    merged_df['user_age_group'] = merged_df['age'].apply(age_group)

    # Process supplement_name and dosage
    merged_df['supplement_name'].fillna('No intake', inplace=True)
    merged_df['dosage_grams'] = merged_df['dosage'].where(merged_df['dosage_unit'] == 'g',
                                                         merged_df['dosage'] / 1000)
    merged_df['sleep_hours'] = merged_df['sleep_hours'].apply(clean_sleep_hours)
    
    # Retain necessary columns and rename as required
    merged_df = merged_df[['user_id', 'date', 'email', 'user_age_group', 'experiment_name', 'supplement_name', 'dosage_grams', 'is_placebo', 'average_heart_rate', 'average_glucose', 'sleep_hours', 'activity_level']]

    # Ensure unique rows for each daily entry
    merged_df = merged_df.groupby(['user_id', 'date', 'email', 'user_age_group', 'supplement_name', 'average_heart_rate', 'average_glucose', 'sleep_hours', 'activity_level'], as_index=False).first()

    merged_df = enforceType(merged_df, 'user_age_group', 'string', False, fill_value='null')
    merged_df = enforceType(merged_df, 'sleep_hours', 'string', False, fill_value='null')
    merged_df['is_placebo'] = merged_df['is_placebo'].astype('bool')
    
    return merged_df

In [9]:
merge_all_data('data/user_health_data.csv', 'data/supplement_usage.csv', 'data/experiments.csv', 'data/user_profiles.csv')

C:\Users\mines\AppData\Local\Temp\ipykernel_48032\467866860.py:20: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[column].fillna(fill_value, inplace=True)
C:\Users\mines\AppData\Local\Temp\ipykernel_48032\467866860.py:94: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example,

,user_id,date,email,user_age_group,supplement_name,average_heart_rate,average_glucose,sleep_hours,activity_level,experiment_name,dosage_grams,is_placebo
0,005dd64e-3863-49f3-93b0-1aea9a84d526,2018-01-31,user_65@myemail.com,26-35,Placebo,84.172177,96.814856,11.4H,2,Memory,0.170989,False
1,005dd64e-3863-49f3-93b0-1aea9a84d526,2018-02-28,user_65@myemail.com,26-35,Magnesium,85.200747,130.836935,5.8H,2,Sleep Quality,0.494938,False
2,005dd64e-3863-49f3-93b0-1aea9a84d526,2018-03-31,user_65@myemail.com,26-35,Placebo,76.695318,109.782341,7.0H,1,Sleep Quality,0.184596,False
3,005dd64e-3863-49f3-93b0-1aea9a84d526,2018-04-30,user_65@myemail.com,26-35,Omega-3,65.772482,100.765545,7.5H,1,Endurance,0.313256,True
4,016e4095-8b5d-43d9-83fd-e28b38be2f7d,2018-01-31,contact_390@email.com,36-45,No intake,86.762141,103.689177,8.2H,2,<NA>,NaN,False
...,...,...,...,...,...,...,...,...,...,...,...,...
2529,ff0e1eb5-1deb-4ddc-9ce9-548706be6bef,2018-04-30,contact_299@myemail.com,26-35,Zinc,78.136649,86.144705,5.3H,2,Memory,0.327864,False
2530,ffaccfef-220a-4976-96f7-22e4f2fc3ecc,2018-01-31,user323@email.com,46-55,Vitamin C,80.122904,134.196895,11.2H,1,Strength,0.407786,False
2531,ffaccfef-220a-4976-96f7-22e4f2fc3ecc,2018-02-28,user323@email.com,46-55,Omega-3,63.316583,79.378241,11.6H,1,Recovery,0.350319,False
2532,ffaccfef-220a-4976-96f7-22e4f2fc3ecc,2018-03-31,user323@email.com,46-55,Zinc,84.800053,117.005827,11.1H,3,Recovery,0.111641,False


In [12]:
def merge_all_data_2(user_health_path, supplements_path, experiments_path, user_profiles_path):
    user_health_df = pd.read_csv(user_health_path)
    supplements_df = pd.read_csv(supplements_path)
    experiments_df = pd.read_csv(experiments_path)
    user_profiles_df = pd.read_csv(user_profiles_path)

    user_health_df['user_id'] = user_health_df['user_id'].apply(lambda x: 'Unknown' if len(x) <= 1 else x)
    user_health_df['date'] = pd.to_datetime(user_health_df['date'],errors='coerce')
    user_health_df['average_heart_rate'] = round(user_health_df['average_heart_rate'].astype(float),2 ).fillna(0)
    user_health_df['average_glucose'] = round(user_health_df['average_glucose'].astype(float),2).fillna(0)
    user_health_df['sleep_hours'] = user_health_df['sleep_hours'].apply(lambda x: x.lower().replace('h', '')).astype(float).fillna(0)
    user_health_df['activity_level'] = user_health_df['activity_level'].fillna(0).astype(int)
    user_health_df = user_health_df[(user_health_df['activity_level'] >= 0) & (user_health_df['activity_level'] <= 100)]

    supplements_df['user_id'] = supplements_df['user_id'].apply(lambda x: 'Unknown' if len(x) <= 1 else x)
    supplements_df['date'] = pd.to_datetime(supplements_df['date'],errors='coerce')
    supplements_df['supplement_name'] = supplements_df['supplement_name'].str.lower()
    supplements_df['supplement_name'] = supplements_df['supplement_name'].apply(lambda x: x.replace(' ', '_').replace('-','_'))
    supplements_df['dosage'] = supplements_df['dosage'].fillna(0)
    supplements_df['dosage_unit'] = supplements_df['dosage_unit'].str.lower()
    supplements_df['dosage_unit'] = supplements_df['dosage_unit'].fillna('Unknown')
    supplements_df.loc[supplements_df['dosage_unit'] == 'mg', 'dosage_grams'] = round(supplements_df['dosage'] / 1000, 2)
    supplements_df['dosage_grams'] = supplements_df['dosage_grams'].fillna(0)
    supplements_df['experiment_id'] = supplements_df['experiment_id'].apply(lambda x: 'undefined' if len(x) <= 1 else x)

    experiments_df['experiment_id'] = experiments_df['experiment_id'].apply(lambda x: 'undefined' if len(x) <= 1 else x)
    experiments_df['name'] = experiments_df['name'].str.lower()
    experiments_df['name'] = experiments_df['name'].apply(lambda x: x.replace(' ', '_').strip())
    experiments_df['name'] = experiments_df['name'].apply(lambda x: 'undefined' if len(x) <= 1 else x)

    user_profiles_df['user_id'] = user_profiles_df['user_id'].apply(lambda x: 'undefined' if len(x) <= 1 else x)
    user_profiles_df['email'] = user_profiles_df['email'].apply(lambda x: x.strip())
    user_profiles_df['email'] = user_profiles_df['email'].apply(lambda x: 'undefined' if len(x) <= 1 else x)
    user_profiles_df['age'] = user_profiles_df['age'].fillna(0).astype(int)

     # Create age groups
    def age_group(age):

        if pd.isna(age):
            return 'Unknown'
        elif age < 18:
            return 'Under 18'
        elif 18 <= age <= 25:
            return '18-25'
        elif 26 <= age <= 35:
            return '26-35'
        elif 36 <= age <= 45:
            return '36-45'
        elif 46 <= age <= 55:
            return '46-55'
        elif 56 <= age <= 65:
            return '56-65'
        else:
            return 'Over 65'
        
    user_profiles_df['user_age_group'] = user_profiles_df['age'].apply(lambda x: age_group(x))


    user_supplements_experiments = pd.merge(supplements_df, experiments_df, on='experiment_id', how="inner", validate="many_to_many")

    user_view = pd.merge(user_health_df, user_supplements_experiments, on=['user_id', 'date'], how='outer', validate="many_to_many")
    user_profile_view = pd.merge(user_view, user_profiles_df, on='user_id', how="inner", validate="many_to_many")

    user_profile_view = user_profile_view[['user_id', 'date', 'email', 'user_age_group', 'name', 'supplement_name', 'dosage_grams', 'is_placebo','average_heart_rate',	'average_glucose', 'sleep_hours', 'activity_level' ]]
    user_profile_view = user_profile_view.rename(columns={'name':'experiment_name'})

    user_profile_view = user_profile_view.dropna(subset=['user_id', 'date', 'email', 'user_age_group'])

    exp_name_missing_values = user_profile_view[
    user_profile_view['experiment_name'].isnull() &
    (user_profile_view['average_heart_rate'].isnull() |
     user_profile_view['average_glucose'].isnull() |
     user_profile_view['sleep_hours'].isnull() |
     user_profile_view['activity_level'].isnull())
    ]

    user_profile_view = user_profile_view.drop(exp_name_missing_values.index)

    user_profile_view['supplement_name'] = user_profile_view['supplement_name'].fillna('No intake')

    is_placebo_mv = user_profile_view[
    user_profile_view['is_placebo'].isnull() &
    (user_profile_view['average_heart_rate'].isnull() |
     user_profile_view['average_glucose'].isnull() |
     user_profile_view['sleep_hours'].isnull() |
     user_profile_view['activity_level'].isnull())
    ]

    user_profile_view = user_profile_view.drop(is_placebo_mv.index)

    user_profile_view = user_profile_view

    return user_profile_view

In [13]:
merge_all_data_2('data/user_health_data.csv', 'data/supplement_usage.csv', 'data/experiments.csv', 'data/user_profiles.csv')

,user_id,date,email,user_age_group,experiment_name,supplement_name,dosage_grams,is_placebo,average_heart_rate,average_glucose,sleep_hours,activity_level
0,005dd64e-3863-49f3-93b0-1aea9a84d526,2018-01-31,user_65@myemail.com,26-35,memory,placebo,0.17,False,84.17,96.81,11.4,2
1,005dd64e-3863-49f3-93b0-1aea9a84d526,2018-02-28,user_65@myemail.com,26-35,sleep_quality,magnesium,0.49,False,85.20,130.84,5.8,2
2,005dd64e-3863-49f3-93b0-1aea9a84d526,2018-03-31,user_65@myemail.com,26-35,sleep_quality,placebo,0.18,False,76.70,109.78,7.0,1
3,005dd64e-3863-49f3-93b0-1aea9a84d526,2018-04-30,user_65@myemail.com,26-35,endurance,omega_3,0.31,True,65.77,100.77,7.5,1
4,016e4095-8b5d-43d9-83fd-e28b38be2f7d,2018-01-31,contact_390@email.com,36-45,NaN,No intake,NaN,NaN,86.76,103.69,8.2,2
...,...,...,...,...,...,...,...,...,...,...,...,...
2716,ff0e1eb5-1deb-4ddc-9ce9-548706be6bef,2018-04-30,contact_299@myemail.com,26-35,endurance,magnesium,0.16,False,78.14,86.14,5.3,2
2717,ffaccfef-220a-4976-96f7-22e4f2fc3ecc,2018-01-31,user323@email.com,46-55,strength,vitamin_c,0.41,False,80.12,134.20,11.2,1
2718,ffaccfef-220a-4976-96f7-22e4f2fc3ecc,2018-02-28,user323@email.com,46-55,recovery,omega_3,0.35,False,63.32,79.38,11.6,1
2719,ffaccfef-220a-4976-96f7-22e4f2fc3ecc,2018-03-31,user323@email.com,46-55,recovery,zinc,0.11,False,84.80,117.01,11.1,3
